## Test of accelerated Monte Carlo ##

In [23]:
import sys
sys.path.insert(0, "../python/")

from particle import particle
from physics import em_physics
from cylinder import cylinder 

import matplotlib.pyplot as plt
import numpy as np
from numba import jit
import pandas as pd

import json
import uproot
import csv

### Define the Geometry ###

In [24]:
# define the geometry
radius = 50 # cm
height = 100 #cm
cryostat = cylinder(R=radius,h=height)

radius = 40 # cm
height = 80 #cm
fiducial = cylinder(R=radius,h=height)

cylinder::__init__ Define cylinder with R= 50  and height= 100
cylinder::__init__ Define cylinder with R= 40  and height= 80


### Setup the photon physics ###

In [25]:
# define the physics
em = em_physics()

### Event generation ###

In [26]:
#%%prun -D my.profile
# number of events
#
nevent = 10000000
#
# gamma energy
#
energy = 2447.86 # keV
#
# output filename
#
#write_mode = 'json'
#write_mode = 'txt'
write_mode = 'csv'


mcout = '../mcdata/testdata' + '.' + write_mode 
#mcout = '../mcdata/mc_reduce_variance' + '.' + write_mode 
#mcout = '../mcdata/mc_plain' + '.' + write_mode 

#
# open output data file
#
f = open(mcout,'w')
if write_mode == 'json':
    f.write('[\n')  
#
# event generation loop
#
tarr = []

for ieve in range(nevent):
    warr=[]
    if ieve%25000 == 0:
        print("generated ",ieve," events")
    #
    # make a particle 'fiducial_scatter'
    #
    p = particle(type='gamma',
                 energy=energy, 
                 geometry=cryostat, 
                 fiducial=fiducial, 
                 vrt=None,
                 edep_max=2600,
                 nscatter_max=2,
                 physics=em,
                 debug=False)
    
    #
    # propagate the particle and retrieve the intersection points
    #
    p.propagate()

    #
    # store data
    #
    if write_mode == 'txt':
        
        wstring = "{:d} {:d} {:7.6e} {:7.6f} {:7.6f} {:7.6f} {:7.6f} ".format(ieve,len(p.xint), 
                                                                                p.weight,p.edep,
                                                                                p.x0start[0],p.x0start[1],p.x0start[2])
        f.write(wstring)
         
    if write_mode == 'csv':   
        wstring = "{:d}, {:d}, {:7.6e}, {:7.6f}, {:7.6f}, {:7.6f}, {:7.6f},".format(ieve,len(p.xint), 
                                                                                p.weight,p.edep,
                                                                                p.x0start[0],p.x0start[1],p.x0start[2])
        
        #warr = [ieve,len(p.xint),p.weight,p.edep,p.x0start[0],p.x0start[1],p.x0start[2]]
        
        f.write(wstring)
        
            
            
    
        for i in range(len(p.xint)):
            #
            # data structure for each energy deposit
            #
            if write_mode == 'json':
                dd = {'x':p.xint[i][0][0],'y':p.xint[i][0][1],'z':p.xint[i][0][2],'de':p.xint[i][1],
                      'w':p.weight,'n':p.nscatter}
                json.dump(dd,f, sort_keys=True, indent=8)
                f.write(',\n')

            else:
                wstring = "{:7.6f}, {:7.6f}, {:7.6f}, {:7.6f},".format(p.xint[i][0][0],
                                                                      p.xint[i][0][1],
                                                                      p.xint[i][0][2],
                                                                      p.xint[i][1])

                #warr.extend([p.xint[i][0][0],p.xint[i][0][1],p.xint[i][0][2],p.xint[i][1]])



                f.write(wstring)

            
    f.write("\n")
    
    #tarr.append(warr)
    
#df= pd.DataFrame(tarr)
#print(df)
#df.to_csv('my_csv.csv', index=False, header=False)


# close output data file
#
if write_mode == 'json':
    f.write('{}]')
elif write_mode == 'txt':
    f.write('-1\n')

    
f.close()

generated  0  events
generated  25000  events
generated  50000  events
generated  75000  events
generated  100000  events
generated  125000  events
generated  150000  events
generated  175000  events
generated  200000  events
generated  225000  events
generated  250000  events
generated  275000  events
generated  300000  events
generated  325000  events
generated  350000  events
generated  375000  events
generated  400000  events
generated  425000  events
generated  450000  events
generated  475000  events
generated  500000  events
generated  525000  events
generated  550000  events
generated  575000  events
generated  600000  events
generated  625000  events
generated  650000  events
generated  675000  events
generated  700000  events
generated  725000  events
generated  750000  events
generated  775000  events
generated  800000  events
generated  825000  events
generated  850000  events
generated  875000  events
generated  900000  events
generated  925000  events
generated  950000  e

generated  7650000  events
generated  7675000  events
generated  7700000  events
generated  7725000  events
generated  7750000  events
generated  7775000  events
generated  7800000  events
generated  7825000  events
generated  7850000  events
generated  7875000  events
generated  7900000  events
generated  7925000  events
generated  7950000  events
generated  7975000  events
generated  8000000  events
generated  8025000  events
generated  8050000  events
generated  8075000  events
generated  8100000  events
generated  8125000  events
generated  8150000  events
generated  8175000  events
generated  8200000  events
generated  8225000  events
generated  8250000  events
generated  8275000  events
generated  8300000  events
generated  8325000  events
generated  8350000  events
generated  8375000  events
generated  8400000  events
generated  8425000  events
generated  8450000  events
generated  8475000  events
generated  8500000  events
generated  8525000  events
generated  8550000  events
g

In [ ]:
!snakeviz my.profile

snakeviz web server started on 127.0.0.1:8080; enter Ctrl-C to exit
http://127.0.0.1:8080/snakeviz/%2FUsers%2Folivier%2FPycharmProjects%2FFastMC%2Fnotebooks%2Fmy.profile


[None, None, None, None, None, None, None, None, None, None]
